Import necessary libraries

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import measure, morphology
from skimage.transform import resize
from scipy.ndimage import binary_closing, binary_opening
from scipy import ndimage
from mayavi import mlab
from skimage import io, filters, exposure
from skimage.restoration import denoise_bilateral
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, Output, VBox
from IPython.display import display

Function to plot 2d slices

In [2]:

def plot_with_ipywidgets(organs_mask, slice_axis=2):
    """
    Visualizza un'immagine volumetrica `organs_mask` con uno slider interattivo
    per cambiare la slice visualizzata.

    Args:
        organs_mask (numpy.ndarray): Volume binario da visualizzare.
        slice_axis (int): Asse lungo il quale effettuare le sezioni (0, 1 o 2).
    """
    # Creare un'area di output separata
    output = Output()

    # Funzione per aggiornare la visualizzazione
    def update(slice_index):
        with output:
            output.clear_output(wait=True)
            plt.figure(figsize=(8, 8))
            if slice_axis == 0:
                slice_data = organs_mask[slice_index, :, :]
            elif slice_axis == 1:
                slice_data = organs_mask[:, slice_index, :]
            else:
                slice_data = organs_mask[:, :, slice_index]
            
            plt.imshow(slice_data, cmap='gray')
            plt.title(f"Slice {slice_index} lungo l'asse {slice_axis}")
            plt.axis('off')
            plt.show()

    # Crea uno slider
    slider = IntSlider(min=0, max=organs_mask.shape[slice_axis] - 1, step=1, value=0)
    slider.observe(lambda change: update(change['new']), names='value')

    # Layout con slider e output
    ui = VBox([slider, output])

    # Inizializza la visualizzazione
    update(0)
    display(ui)

# Esempio di utilizzo




Function to filter lungs

In [3]:
def filtra_polmoni(data_resized,voxel_dimensions):
    organs_mask = (data_resized < -500) & (data_resized > -1000)
    #plot_with_ipywidgets(organs_mask, slice_axis=2)
    organs_mask_closed=np.zeros_like(organs_mask)
    organs_mask_cleaned=np.zeros_like(organs_mask)
    for i in range(organs_mask.shape[2]):
        # Applica prima una chiusura morfologica per eliminare piccoli buchi
        selem = morphology.disk(3)  # Elemento strutturante sferico di raggio 3
        organs_mask_closed[:,:,i] = binary_closing(organs_mask[:,:,i], structure=selem)
        #plot_with_ipywidgets(organs_mask_closed, slice_axis=2)
        
        # Applica un'apertura morfologica per rimuovere piccole protuberanze
        organs_mask_cleaned[:,:,i] = binary_opening(organs_mask_closed[:,:,i], structure=selem)
    #plot_with_ipywidgets(organs_mask_cleaned, slice_axis=2)
    
    labeled_volume, num_features = ndimage.label(organs_mask_cleaned)
    print(f"Numero di blob trovati: {num_features}")

    # Genera la mappa di colori per distinguere i blob
    colors = cm.rainbow(np.linspace(0, 1, num_features))
    mlab.figure(bgcolor=(1, 1, 1))
    blob_properties = measure.regionprops(labeled_volume)

    filtrati = []
    for i, prop in enumerate(blob_properties):
        if prop.area > 100000:
            filtrati.append((prop.area,i + 1))
    filtrati=sorted(filtrati)
    filtrati=filtrati[0:2]
    for (area,blob_idx) in filtrati:
        blob = (labeled_volume == blob_idx)
        verts, faces, _, _ = measure.marching_cubes(blob, level=0.5)

        # Scala le coordinate dei vertici in base ai voxel
        verts[:, 0] *= voxel_dimensions[0]
        verts[:, 1] *= voxel_dimensions[1]
        verts[:, 2] *= voxel_dimensions[2]

        # Colore per il blob corrente
        color = tuple(colors[blob_idx - 1][:3])
        mlab.triangular_mesh(verts[:, 0], verts[:, 1], verts[:, 2], faces, color=color)

    mlab.axes(extent=[0, data_resized.shape[0] * voxel_dimensions[0],
                      0, data_resized.shape[1] * voxel_dimensions[1],
                      0, data_resized.shape[2] * voxel_dimensions[2]])
    mlab.show()

Read .nii MRI image

In [4]:
# Carica il file .nii
img = nib.load('estratto3.nii')
data = img.get_fdata()
# Ottieni informazioni sulla dimensione dei voxel
header = img.header
voxel_dimensions = header.get_zooms()


Launch filter on image

In [5]:
filtra_polmoni(data,voxel_dimensions)

Numero di blob trovati: 40
